# Stock NeurIPS2018 Part 1. Data
This series is a reproduction of paper *the process in the paper Practical Deep Reinforcement Learning Approach for Stock Trading*. 

This is the first part of the NeurIPS2018 series, introducing how to use FinRL to fetch and process data that we need for ML/RL trading.

Other demos can be found at the repo of [FinRL-Tutorials]((https://github.com/AI4Finance-Foundation/FinRL-Tutorials)).

# Part 1. Install Packages

In [1]:
import pandas as pd
import numpy as np
import datetime
import yfinance as yf

from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl import config_tickers
from finrl.config import INDICATORS

import itertools

# Part 2. Fetch data

[yfinance](https://github.com/ranaroussi/yfinance) is an open-source library that provides APIs fetching historical data form Yahoo Finance. In FinRL, we have a class called [YahooDownloader](https://github.com/AI4Finance-Foundation/FinRL/blob/master/finrl/meta/preprocessor/yahoodownloader.py) that use yfinance to fetch data from Yahoo Finance.

**OHLCV**: Data downloaded are in the form of OHLCV, corresponding to **open, high, low, close, volume,** respectively. OHLCV is important because they contain most of numerical information of a stock in time series. From OHLCV, traders can get further judgement and prediction like the momentum, people's interest, market trends, etc.

## Data for a single ticker

Here we provide two ways to fetch data with single ticker, let's take Apple Inc. (AAPL) as an example.

### Using yfinance

In [2]:
aapl_df_yf = yf.download('AAPL', period='max')

[*********************100%***********************]  1 of 1 completed


In [3]:
len(aapl_df_yf)

11084

In [4]:
aapl_df_yf.head()

Price,Adj Close,Close,High,Low,Open,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL,AAPL
Date,,,,,,
1980-12-12,0.098834,0.128348,0.128906,0.128348,0.128348,469033600
1980-12-15,0.093678,0.121652,0.122210,0.121652,0.122210,175884800
1980-12-16,0.086802,0.112723,0.113281,0.112723,0.113281,105728000
1980-12-17,0.088951,0.115513,0.116071,0.115513,0.115513,86441600
1980-12-18,0.091530,0.118862,0.119420,0.118862,0.118862,73449600


In [5]:
aapl_df_yf.tail()

Price,Adj Close,Close,High,Low,Open,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL,AAPL
Date,,,,,,
2024-11-22,229.869995,229.869995,230.720001,228.059998,228.059998,38168300
2024-11-25,232.869995,232.869995,233.250000,229.740005,231.460007,90152800
2024-11-26,235.059998,235.059998,235.570007,233.330002,233.330002,45986200
2024-11-27,234.929993,234.929993,235.690002,233.809998,234.470001,33498400
2024-11-29,237.330002,237.330002,237.809998,233.970001,234.809998,28481400


### Using FinRL

In FinRL's YahooDownloader, we modified the data frame to the form that convenient for further data processing process. We use adjusted close price instead of close price, and add a column representing the day of a week (0-4 corresponding to Monday-Friday).

In [7]:
aapl_df_finrl = YahooDownloader(ticker_list=['AAPL'],start_date='1980-12-12',end_date='2022-11-29').fetch_data()

[*********************100%***********************]  1 of 1 completed

Shape of DataFrame:  (10580, 8)


## Data for the chosen tickers

In [8]:
config_tickers.DOW_30_TICKER

['AXP',
 'AMGN',
 'AAPL',
 'BA',
 'CAT',
 'CSCO',
 'CVX',
 'GS',
 'HD',
 'HON',
 'IBM',
 'INTC',
 'JNJ',
 'KO',
 'JPM',
 'MCD',
 'MMM',
 'MRK',
 'MSFT',
 'NKE',
 'PG',
 'TRV',
 'UNH',
 'CRM',
 'VZ',
 'V',
 'WBA',
 'WMT',
 'DIS',
 'DOW']

In [10]:
TRAIN_START_DATE = '1989-01-01'
TRAIN_END_DATE = '2022-07-01'
TRADE_START_DATE = '2022-07-01'
TRADE_END_DATE = '2024-10-29'

In [11]:
df_raw = YahooDownloader(start_date=TRAIN_START_DATE, end_date=TRADE_END_DATE,
                         ticker_list=config_tickers.DOW_30_TICKER).fetch_data()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

Shape of DataFrame:  (251497, 8)


In [12]:
df_raw.head(100)

,date,open,high,low,close,volume,tic,day
0,1989-01-03,0.281357,0.360491,0.361607,0.359375,100016000,AAPL,1
1,1989-01-03,0.468830,0.666667,0.703125,0.703125,17054400,AMGN,1
2,1989-01-03,3.686327,6.855514,6.887699,6.887699,3482157,AXP,1
3,1989-01-03,6.973380,13.250000,13.500000,13.472222,2060550,BA,1
4,1989-01-03,3.399682,7.953125,7.953125,7.953125,2290400,CAT,1
...,...,...,...,...,...,...,...,...
95,1989-01-06,3.997490,10.812500,10.875000,10.812500,783600,TRV,4
96,1989-01-06,0.105532,0.132813,0.140625,0.000000,243200,UNH,4
97,1989-01-06,3.073664,15.850564,15.906771,15.738148,536404,VZ,4
98,1989-01-06,0.966001,1.976563,1.992188,1.968750,4168000,WBA,4


# Part 3: Preprocess Data
We need to check for missing data and do feature engineering to convert the data point into a state.
* **Adding technical indicators**. In practical trading, various information needs to be taken into account, such as historical prices, current holding shares, technical indicators, etc. Here, we demonstrate two trend-following technical indicators: MACD and RSI.
* **Adding turbulence index**. Risk-aversion reflects whether an investor prefers to protect the capital. It also influences one's trading strategy when facing different market volatility level. To control the risk in a worst-case scenario, such as financial crisis of 2007–2008, FinRL employs the turbulence index that measures extreme fluctuation of asset price.

Hear let's take MACD as an example. Moving average convergence/divergence (MACD) is one of the most commonly used indicator showing bull and bear market. Its calculation is based on EMA (Exponential Moving Average indicator, measuring trend direction over a period of time.)

In [16]:
print(df_raw.isnull())
print(np.isinf(df_raw))

         date   open   high    low  close  volume    tic    day
0       False  False  False  False  False   False  False  False
1       False  False  False  False  False   False  False  False
2       False  False  False  False  False   False  False  False
3       False  False  False  False  False   False  False  False
4       False  False  False  False  False   False  False  False
...       ...    ...    ...    ...    ...     ...    ...    ...
251492  False  False  False  False  False   False  False  False
251493  False  False  False  False  False   False  False  False
251494  False  False  False  False  False   False  False  False
251495  False  False  False  False  False   False  False  False
251496  False  False  False  False  False   False  False  False

[251497 rows x 8 columns]


TypeError: ufunc 'isinf' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [17]:
df_raw.dropna(inplace=True)

In [19]:
fe = FeatureEngineer(use_technical_indicator=True,
                     tech_indicator_list=INDICATORS,
                     use_vix=True,
                     use_turbulence=False,
                     user_defined_feature=False)
processed = fe.preprocess_data(df_raw)

[*********************100%***********************]  1 of 1 completed

Successfully added technical indicators
Shape of DataFrame:  (8772, 8)
Successfully added vix


In [20]:
list_ticker = processed["tic"].unique().tolist()
list_ticker

['AAPL',
 'AMGN',
 'AXP',
 'BA',
 'CAT',
 'CVX',
 'DIS',
 'HD',
 'HON',
 'IBM',
 'INTC',
 'JNJ',
 'JPM',
 'KO',
 'MCD',
 'MMM',
 'MRK',
 'MSFT',
 'NKE',
 'PG',
 'TRV',
 'UNH',
 'VZ',
 'WBA',
 'WMT']

In [21]:
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
list_date

['1990-01-02',
 '1990-01-03',
 '1990-01-04',
 '1990-01-05',
 '1990-01-06',
 '1990-01-07',
 '1990-01-08',
 '1990-01-09',
 '1990-01-10',
 '1990-01-11',
 '1990-01-12',
 '1990-01-13',
 '1990-01-14',
 '1990-01-15',
 '1990-01-16',
 '1990-01-17',
 '1990-01-18',
 '1990-01-19',
 '1990-01-20',
 '1990-01-21',
 '1990-01-22',
 '1990-01-23',
 '1990-01-24',
 '1990-01-25',
 '1990-01-26',
 '1990-01-27',
 '1990-01-28',
 '1990-01-29',
 '1990-01-30',
 '1990-01-31',
 '1990-02-01',
 '1990-02-02',
 '1990-02-03',
 '1990-02-04',
 '1990-02-05',
 '1990-02-06',
 '1990-02-07',
 '1990-02-08',
 '1990-02-09',
 '1990-02-10',
 '1990-02-11',
 '1990-02-12',
 '1990-02-13',
 '1990-02-14',
 '1990-02-15',
 '1990-02-16',
 '1990-02-17',
 '1990-02-18',
 '1990-02-19',
 '1990-02-20',
 '1990-02-21',
 '1990-02-22',
 '1990-02-23',
 '1990-02-24',
 '1990-02-25',
 '1990-02-26',
 '1990-02-27',
 '1990-02-28',
 '1990-03-01',
 '1990-03-02',
 '1990-03-03',
 '1990-03-04',
 '1990-03-05',
 '1990-03-06',
 '1990-03-07',
 '1990-03-08',
 '1990-03-

In [22]:
combination = list(itertools.product(list_date,list_ticker))
combination

[('1990-01-02', 'AAPL'),
 ('1990-01-02', 'AMGN'),
 ('1990-01-02', 'AXP'),
 ('1990-01-02', 'BA'),
 ('1990-01-02', 'CAT'),
 ('1990-01-02', 'CVX'),
 ('1990-01-02', 'DIS'),
 ('1990-01-02', 'HD'),
 ('1990-01-02', 'HON'),
 ('1990-01-02', 'IBM'),
 ('1990-01-02', 'INTC'),
 ('1990-01-02', 'JNJ'),
 ('1990-01-02', 'JPM'),
 ('1990-01-02', 'KO'),
 ('1990-01-02', 'MCD'),
 ('1990-01-02', 'MMM'),
 ('1990-01-02', 'MRK'),
 ('1990-01-02', 'MSFT'),
 ('1990-01-02', 'NKE'),
 ('1990-01-02', 'PG'),
 ('1990-01-02', 'TRV'),
 ('1990-01-02', 'UNH'),
 ('1990-01-02', 'VZ'),
 ('1990-01-02', 'WBA'),
 ('1990-01-02', 'WMT'),
 ('1990-01-03', 'AAPL'),
 ('1990-01-03', 'AMGN'),
 ('1990-01-03', 'AXP'),
 ('1990-01-03', 'BA'),
 ('1990-01-03', 'CAT'),
 ('1990-01-03', 'CVX'),
 ('1990-01-03', 'DIS'),
 ('1990-01-03', 'HD'),
 ('1990-01-03', 'HON'),
 ('1990-01-03', 'IBM'),
 ('1990-01-03', 'INTC'),
 ('1990-01-03', 'JNJ'),
 ('1990-01-03', 'JPM'),
 ('1990-01-03', 'KO'),
 ('1990-01-03', 'MCD'),
 ('1990-01-03', 'MMM'),
 ('1990-01-03', '

In [23]:
processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])

processed_full = processed_full.fillna(0)

In [24]:
processed_full

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix
0,1990-01-02,AAPL,0.262129,0.332589,0.334821,0.314732,183198400.0,1.0,-0.020410,0.407460,0.269995,37.182693,-57.773632,14.795458,0.358333,0.385603,17.240000
1,1990-01-02,AMGN,0.754523,1.072917,1.072917,1.026042,19267200.0,1.0,-0.037307,1.269225,0.980255,44.088693,-108.941827,10.538710,1.152865,1.136372,17.240000
2,1990-01-02,AXP,4.953164,8.979757,9.011943,8.915386,4301237.0,1.0,-0.113536,9.354541,7.880800,50.163415,44.429024,19.240226,8.751240,9.057539,17.240000
3,1990-01-02,BA,11.043628,20.500000,20.500000,19.791668,2042400.0,1.0,-0.012113,20.580973,18.739861,53.656871,109.434371,42.703571,19.641667,19.314583,17.240000
4,1990-01-02,CAT,3.208576,7.359375,7.359375,7.234375,2910400.0,1.0,-0.045713,7.750962,6.961538,47.910104,-16.224189,0.407785,7.323958,7.242448,17.240000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
317895,2024-10-25,TRV,250.500000,250.500000,257.399994,257.309998,942300.0,4.0,7.545260,270.832537,217.118464,61.945197,97.029254,18.669475,242.373000,231.965833,19.219999
317896,2024-10-25,UNH,564.559998,564.559998,571.500000,566.159973,2513700.0,4.0,-4.788629,609.836457,548.415533,48.361730,-93.539029,10.035612,579.122329,581.195831,19.219999
317897,2024-10-25,VZ,41.380001,41.380001,42.180000,41.900002,19717400.0,4.0,-0.299102,45.693344,41.572658,46.303429,-216.077122,5.430581,43.884667,42.602334,19.219999
317898,2024-10-25,WBA,8.987005,9.260000,9.570000,9.370000,18012300.0,4.0,0.185967,10.997480,7.938520,45.245095,10.708690,2.872925,9.250333,9.634833,19.219999


# Part 4: Save the Data

### Split the data for training and trading

In [25]:
train = data_split(processed_full, TRAIN_START_DATE, TRAIN_END_DATE)
train

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix
0,1990-01-02,AAPL,0.262129,0.332589,0.334821,0.314732,183198400.0,1.0,-0.020410,0.407460,0.269995,37.182693,-57.773632,14.795458,0.358333,0.385603,17.24
0,1990-01-02,AMGN,0.754523,1.072917,1.072917,1.026042,19267200.0,1.0,-0.037307,1.269225,0.980255,44.088693,-108.941827,10.538710,1.152865,1.136372,17.24
0,1990-01-02,AXP,4.953164,8.979757,9.011943,8.915386,4301237.0,1.0,-0.113536,9.354541,7.880800,50.163415,44.429024,19.240226,8.751240,9.057539,17.24
0,1990-01-02,BA,11.043628,20.500000,20.500000,19.791668,2042400.0,1.0,-0.012113,20.580973,18.739861,53.656871,109.434371,42.703571,19.641667,19.314583,17.24
0,1990-01-02,CAT,3.208576,7.359375,7.359375,7.234375,2910400.0,1.0,-0.045713,7.750962,6.961538,47.910104,-16.224189,0.407785,7.323958,7.242448,17.24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8187,2022-06-30,TRV,161.338013,169.130005,169.500000,164.610001,1336100.0,3.0,-2.374215,180.224943,156.931054,44.639520,-44.912598,8.641378,170.339999,173.666833,29.42
8187,2022-06-30,UNH,497.025818,513.630005,516.320007,513.549988,3568900.0,3.0,3.243815,522.217561,447.697438,54.941784,134.358930,26.659363,486.779667,500.980667,29.42
8187,2022-06-30,VZ,43.047783,50.750000,51.220001,50.669998,18880500.0,3.0,0.170573,52.597046,48.530953,50.161884,21.317367,13.348827,50.415666,50.484500,29.42
8187,2022-06-30,WBA,31.923557,37.900002,39.669998,39.520000,15611700.0,3.0,-0.682825,44.006490,38.629509,39.126940,-160.885298,26.648574,41.693666,43.023000,29.42


In [26]:
trade = data_split(processed_full, TRADE_START_DATE, TRADE_END_DATE)
trade

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix
0,2022-07-01,AAPL,137.066147,138.929993,139.039993,136.039993,71051600.0,4.0,-2.506138,150.354667,127.362333,42.668917,-44.566334,2.993118,140.091666,150.219833,29.530001
0,2022-07-01,AMGN,227.034637,245.550003,246.000000,243.250000,2745500.0,4.0,-0.201872,251.918793,231.317207,50.801379,-7.106816,16.362315,244.750333,244.900167,29.530001
0,2022-07-01,AXP,136.196823,140.399994,140.889999,138.070007,2625300.0,4.0,-6.402676,171.878342,128.274658,39.107922,-101.181040,23.290635,153.768333,164.204167,29.530001
0,2022-07-01,BA,139.839996,139.839996,141.789993,135.880005,7627100.0,4.0,-0.173603,147.371193,121.357809,45.537900,68.296219,1.123293,131.727001,145.710167,29.530001
0,2022-07-01,CAT,169.630936,178.289993,181.320007,179.309998,3423100.0,4.0,-8.420835,234.739145,168.980853,37.696671,-137.268271,31.974925,204.527999,210.235999,29.530001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
583,2024-10-25,TRV,250.500000,250.500000,257.399994,257.309998,942300.0,4.0,7.545260,270.832537,217.118464,61.945197,97.029254,18.669475,242.373000,231.965833,19.219999
583,2024-10-25,UNH,564.559998,564.559998,571.500000,566.159973,2513700.0,4.0,-4.788629,609.836457,548.415533,48.361730,-93.539029,10.035612,579.122329,581.195831,19.219999
583,2024-10-25,VZ,41.380001,41.380001,42.180000,41.900002,19717400.0,4.0,-0.299102,45.693344,41.572658,46.303429,-216.077122,5.430581,43.884667,42.602334,19.219999
583,2024-10-25,WBA,8.987005,9.260000,9.570000,9.370000,18012300.0,4.0,0.185967,10.997480,7.938520,45.245095,10.708690,2.872925,9.250333,9.634833,19.219999


### Save data to csv file



In [27]:
train.to_csv(f'train_data_dow30_{TRAIN_START_DATE}_{TRAIN_END_DATE}.csv')
trade.to_csv(f'trade_data_dow30_{TRADE_START_DATE}_{TRADE_END_DATE}.csv')